In [3]:
import sklearn
import math
import random
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor

## Load data

In [53]:
## 读取文件夹下多个数据
def load_data(file_dir):
    pos = []
    energy = []
    file_list = os.listdir(file_dir) ##文件列表
    for i1 in file_list:
        # orig_os = os.getcwd()  ##返回当前的工作目录 
        pos_tmp = []
        energy_tmp = []
        os.chdir(file_dir)    ## ./data/Au20_OPT_1000/ 更改工作目录
        with open(i1,'r') as df:
            dfread = df.read()
            match_tmp1 = re.findall(r"\n.*", dfread)##正则化匹配，.表示匹配除了\n之外的内容，返回的是一个列表
            energy_tmp.append(float(match_tmp1[0]))
            match_tmp2 = re.findall(r"Au.*", dfread)
            # print(match_tmp2)
            for pop_i in range(len(match_tmp2)):
                xyz = match_tmp2[pop_i].split("    ")
                x, y, z = float(xyz[1]), float(xyz[2]), float(xyz[3])
                pos_tmp.append([x, y, z])
        # os.chdir(orig_os)
        pos = pos + pos_tmp ##每个文件的数据接在一起
        energy = energy + energy_tmp 
    pos = np.array(pos)
    energy = np.array(energy)
    return pos, energy

In [58]:
file_dir_Au = 'D:/Vs_projects/MathorCup/origin_data/Au20_OPT_1000'
# file_dir_Au = 'D:/Vs_projects/MathorCup/test_data'
pos_tmp_Au, energy_Au = load_data(file_dir_Au)# 
pos_Au = pos_tmp_Au.reshape(-1, 20, 3)

## Parameter selection for Sutton-Chen potential energy function

In [60]:
def SC_potential_energy_func(x, a): ##需要训练的势能函数
    n = 10
    m = 8
    c = 34.408
    # a = 1
    epsilon = 1

    dis_mat = np.array(dist_matric(x))
    tmp1 = (dis_mat / a + np.eye(dis_mat.shape[0])) ** (-n) - np.eye(dis_mat.shape[0])
    tmp2 = (dis_mat / a + np.eye(dis_mat.shape[0])) ** (-m) - np.eye(dis_mat.shape[0])
    vr = 1 / 2 * (np.sum(tmp1, axis=1))
    vd = -c * np.sqrt(np.sum(tmp2, axis=1))
    energy = epsilon * np.sum(vr + vd)
    return energy

def dist_matric(pos):    
    dis_mat=[]
    for def_i in range(0,len(pos)):
        dis_mat.append([])
        for def_j in range(0,len(pos)):
            temp=math.sqrt(math.pow(pos[def_j][0]-pos[def_i][0],2)+math.pow(pos[def_j][1]-pos[def_i][1],2)+math.pow(pos[def_j][2]-pos[def_i][2],2))
            dis_mat[def_i].append(temp)
    return(dis_mat)

In [61]:
def criterion(label,pred):
    return np.mean(np.abs(label - pred))

def parameter_selection(data, para_grid):
    error_total = []
    for para in para_grid:
        error_l = 0
        for i in range(data.shape[0]):
            ene = SC_potential_energy_func(data[i],para)
            err = criterion(energy_Au[i],ene)
            error_l += err
        error_total.append(error_l)
    error_total = np.array(error_total)
    indx = np.argmin(error_total)
    return para_grid[indx], np.min(error_total/data.shape[0])

In [62]:
Au_a = np.linspace(1, 3, 200)
best_Au_a, error_Au_train = parameter_selection(pos_Au[:800], Au_a)
print("Best parameter for Au20:", best_Au_a)
print("Train error:", error_Au_train)
_, error_Au_test = parameter_selection(pos_Au[800:], [best_Au_a])
print("Test error:", error_Au_test)

Best parameter for Au20: 2.698492462311558
Train error: 78.99850092276058


In [10]:
def SC_energy_func(x, idx):  ##训练好的势能函数
    n = 10
    m = 8
    c = 34.408
    a = best_Au_a
    # epsilon = best_Au_epsilon
    epsilon = 1
    x = np.array(x)
    new_Au_pos_ = np.vstack((new_Au_pos[:idx], np.vstack((x, new_Au_pos[idx+1:]))))
    new_dis = np.array(dist_matric(new_Au_pos_))
    new_dis_diag = np.diag(new_dis)
    tmp1 = (new_dis / a + np.eye(new_dis.shape[0])) ** (-n) - np.eye(new_dis.shape[0])
    tmp2 = (new_dis / a + np.eye(new_dis.shape[0])) ** (-m) - np.eye(new_dis.shape[0])
    vr = 1 / 2 * (np.sum(tmp1, axis=1))
    vd = -c * np.sqrt(np.sum(tmp2, axis=1))
    energy = epsilon * np.sum(vr + vd)
    return energy

## Topology Feature Generation

In [ ]:
def dist_matric(pos):    
    dis_mat=[]
    for def_i in range(0,len(pos)):
        dis_mat.append([])
        for def_j in range(0,len(pos)):
            temp=math.sqrt(math.pow(pos[def_j][0]-pos[def_i][0],2)+math.pow(pos[def_j][1]-pos[def_i][1],2)+math.pow(pos[def_j][2]-pos[def_i][2],2))
            dis_mat[def_i].append(temp)
    return(dis_mat)



In [ ]:
def pos2betti(pos):
    betti=[]
    for i1 in range(len(pos)):
        dis_m=dist_matric(pos[i1])
        ripser_tmp=ripser(np.array(dis_m),maxdim=2,distance_matrix=True)
        ripser_tmp2=ripser_tmp.get('dgms')
        betti0_tmp=ripser_tmp2[0]
        betti1_tmp=ripser_tmp2[1]
        betti2_tmp=ripser_tmp2[2]
        betti0=[]
        betti1=[]
        betti2=[]
        betti3=[]
        betti0_ini=[]
        betti1_ini=[]
        betti2_ini=[]
        betti0_fin=[]
        betti1_fin=[]
        betti2_fin=[]
        i_thresh_mat=list(map(lambda x:x/far_x,range(0,far_x*10+1,1)))
        for i_thresh in range(len(i_thresh_mat)-1):
            betti0.append(0)
            betti1.append(0)
            betti2.append(0)
            betti3.append(0)
            betti0_ini.append(0)
            betti1_ini.append(0)
            betti2_ini.append(0)
            betti0_fin.append(0)
            betti1_fin.append(0)
            betti2_fin.append(0)
            for i2 in range(len(betti0_tmp)):
                if i_thresh_mat[i_thresh] >= betti0_tmp[i2][0] and betti0_tmp[i2][1]>=i_thresh_mat[i_thresh+1]:
                    betti0[i_thresh]=betti0[i_thresh]+1
                elif i_thresh_mat[i_thresh] < betti0_tmp[i2][1] and betti0_tmp[i2][1]<i_thresh_mat[i_thresh+1]:
                    betti0[i_thresh]=betti0[i_thresh]+1
            for i3 in range(len(betti1_tmp)):
                if (i_thresh_mat[i_thresh] >= betti1_tmp[i3][0] and betti1_tmp[i3][1]>=i_thresh_mat[i_thresh+1]):
                    betti1[i_thresh]=betti1[i_thresh]+1
                elif (i_thresh_mat[i_thresh] < betti1_tmp[i3][0] and betti1_tmp[i3][1]<i_thresh_mat[i_thresh+1]):
                    betti1[i_thresh]=betti1[i_thresh]+1
                elif (i_thresh_mat[i_thresh] < betti1_tmp[i3][0] and betti1_tmp[i3][0]<i_thresh_mat[i_thresh+1]):
                    betti1[i_thresh]=betti1[i_thresh]+1
                elif (i_thresh_mat[i_thresh] < betti1_tmp[i3][1] and betti1_tmp[i3][1]<i_thresh_mat[i_thresh+1]):
                    betti1[i_thresh]=betti1[i_thresh]+1
            for i4 in range(len(betti2_tmp)):
                if i_thresh_mat[i_thresh] >= betti2_tmp[i4][0] and betti2_tmp[i4][1]>=i_thresh_mat[i_thresh+1]:
                    betti2[i_thresh]=betti2[i_thresh]+1
                elif i_thresh_mat[i_thresh] < betti2_tmp[i4][0] and betti2_tmp[i4][1]<i_thresh_mat[i_thresh+1]:
                    betti2[i_thresh]=betti2[i_thresh]+1
                elif i_thresh_mat[i_thresh] < betti2_tmp[i4][0] and betti2_tmp[i4][0]<i_thresh_mat[i_thresh+1]:
                    betti2[i_thresh]=betti2[i_thresh]+1
                elif i_thresh_mat[i_thresh] < betti2_tmp[i4][1] and betti2_tmp[i4][1]<i_thresh_mat[i_thresh+1]:
                    betti2[i_thresh]=betti2[i_thresh]+1

            for i5 in range(len(dis_m)-1):
                for i6 in range(i5+1,len(dis_m)):
                    if i_thresh_mat[i_thresh] < dis_m[i5][i6] <=i_thresh_mat[i_thresh+1] :
                        betti3[i_thresh]=betti3[i_thresh]+1
        betti.append(betti0+betti1+betti2+betti3)
    return np.array(betti)

In [ ]:
data = pos_Au 
energy = energy_Au
far_x=10
betti0 = pos2betti(data)
print(len(betti0[0]
train_data = betti0[:800]
test_data = betti0[800:]
test_data_SC= data[800:]
train_label = energy[:800]
test_label = energy[800:]))

In [ ]:
# train_data = total_data[:800]
#  test_data = total_data8700:]# # train_label = total_label8:700# 
# test_label = total_lab8l[70
train_data = betti0[:800]
test_data = betti0[800:]
test_data_SC= data[800:]
train_label = energy[:800]
test_label = energy[800:]0# :]
train_data = total_data[:25# 00]
test_data = total_data[25# 00:]
train_label = total_label[:# 2500]
test_label = total_label[2500:]
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
print("Train label shape:", train_label.shape)
print("Train label shape:", test_label.shape)

In [ ]:
## 画出一个结构的barcode图
plot_barcode(data[4], "BarCode.png")

In [ ]:
## 建立模型

neigh = KNeighborsRegressor(n_neighbors=20)
knn = neigh.fit(train_data, train_label)
pred = knn.predict(test_data)
error_bar = abs(pred - test_label)
gbr = GradientBoostingRegressor(loss = 'ls',
                                learning_rate = 0.1,
                                n_estimators = 20,
                                subsample = 0.6, 
                                criterion = 'friedman_mse',
                                min_samples_split = 50,
                                max_depth = 3,
                                random_state = 20,
                                alpha = 0.9,
                                verbose = 0)
model = gbr.fit(train_data, train_label)
pred_gbr = model.predict(test_data)
err_bar_gbr = abs(pred_gbr - test_label)
# mse = sklearn.metrics.mean_squared_error(test_label, pred_gbr, sample_weight=None, multioutput='uniform_average')
# mae = sklearn.metrics.mean_absolute_error(test_label, pred_gbr, sample_weight=None, multioutput='uniform_average')
# print("MSE:", mse)
# print("MAE:", mae)
error_SC = []
test_data_SC = data[800:]
a = Au_a 
for i in range(len(test_label)):
    error_SC.append(abs(test_label[i] - SC_potential_energy_func(test_data_SC[i], a)))



In [ ]:
fig = plt.figure(figsize=(25,9),dpi = 200)
plot_result = {"KNN":error_bar,"GBR":err_bar_gbr}
plot_result = pd.DataFrame(plot_result)
plot_result.plot.box()
plt.ylabel("MAE")
plt.savefig("ML_Box.png")
plt.show()


In [ ]:
plt.boxplot(error_SC)
plt.ylabel("MAE")
plt.xticks(ticks=[1],labels=["Sutton-Chen"])
plt.savefig("SC_Box.png")
plt.show()


## PSO

In [36]:
class Particle:
    def __init__(self,x0, idx):
        self.idx = idx
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i = -1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i, self.idx)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self, costFunc, x0, idx, bounds, num_particles, maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        self.err_best_g = -1                   # best error for group                   # best position for group
        self.maxiter = maxiter
        self.num_particles = num_particles
        self.bounds = bounds
        self.idx = idx
        self.x0 = x0
        self.costFunc = costFunc

    
    def pso(self):
        pos_best_g=[]
        # establish the swarm
        swarm=[]
        for i in range(0,self.num_particles):
            swarm.append(Particle(self.x0, self.idx))

        # begin optimization loop
        i=0
        while i < self.maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,self.num_particles):
                swarm[j].evaluate(self.costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < self.err_best_g or self.err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    self.err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,self.num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(self.bounds)
            i+=1
        return pos_best_g, self.err_best_g

In [72]:
pos_Au_init = pos_Au[0]
new_Au_pos = pos_Au_init
energy_Au_init = SC_energy_func(pos_Au_init[0], 0)
print("orig pos:\n", pos_Au_init)
print("orig energy:\n", energy_Au_init)
print("real energy:\n", energy_Au[0])
### set bounds
x_max_Au = np.max(pos_Au[:,:,0])
x_min_Au = np.min(pos_Au[:,:,0])
y_max_Au = np.max(pos_Au[:,:,1])
y_min_Au = np.min(pos_Au[:,:,1])
z_max_Au = np.max(pos_Au[:,:,2])
z_min_Au = np.min(pos_Au[:,:,2])
x_Au_bounds = (x_min_Au, x_max_Au)
y_Au_bounds = (y_min_Au, y_max_Au)
z_Au_bounds = (z_min_Au, z_max_Au)
pos_Au_final = []
err_Au_final = []

Au_bounds = [x_Au_bounds, y_Au_bounds, z_Au_bounds]
epochs = 10
for epoch in range(epochs):
    pos_Au_final_tmp = []
    err_Au_final_tmp = []
    for i in range(pos_Au_init.shape[0]):
        if epoch == 0 and i == 0:
            new_Au_pos = pos_Au_init
        else:
            new_Au_pos = new_Au_pos_
        pos_Au_best_g_tmp, err_Au_best_g_tmp = PSO(SC_energy_func, new_Au_pos[i], i, Au_bounds, num_particles=15, maxiter=30).pso()
        new_Au_pos_ = np.vstack((new_Au_pos[:i], np.vstack((pos_Au_best_g_tmp, new_Au_pos[i+1:]))))
        pos_Au_final_tmp.append(pos_Au_best_g_tmp)
        err_Au_final_tmp.append(err_Au_best_g_tmp)
    pos_Au_final.append(pos_Au_final_tmp)
    err_Au_final.append(err_Au_final_tmp)

In [15]:
print("Best struction for Au20:\n", np.array(pos_Au_final)[-1])
print("Lowest energy for Au20:\n", np.array(err_Au_final)[-1][-1])

Best struction:
 [[ 3.30806347e+00  6.44267281e-01 -1.03140073e+00]
 [-4.96130384e-01  4.13737753e-02 -5.45863977e-01]
 [ 5.41751593e-01  3.30380907e+00 -1.12159112e+00]
 [-1.22170497e+00 -2.72246291e+00 -8.31325269e-02]
 [-3.32234680e+00  1.21969957e-03 -1.33465928e+00]
 [ 3.88401430e+00 -9.52961047e-01 -1.15336128e+00]
 [ 1.19322663e-01 -3.24069072e+00 -1.02342374e+00]
 [-1.06700428e+00  3.15712666e+00 -5.81425001e-01]
 [-1.98813622e+00 -8.67697965e-01 -7.53963637e-01]
 [ 2.50765249e+00 -6.11244662e-01 -2.09227125e-01]
 [-4.29616745e-01 -1.59804464e+00 -1.13063477e+00]
 [-2.14067512e-01  1.71885963e+00 -9.26553926e-01]
 [ 4.19565696e-01  2.37255104e-01  4.28971343e+00]
 [-4.86536217e-01 -8.00368944e-01  5.28905421e+00]
 [ 6.54926269e-01  2.59263334e-01  5.97510041e+00]
 [-7.85044982e-01  8.74986562e-01  5.30882017e+00]
 [-1.90811488e+00  8.71287346e-01 -1.08974944e+00]
 [ 4.04633198e+00 -9.97801557e-02  3.09777272e-01]
 [-8.13958384e-02 -3.77605328e+00  5.29390279e-01]
 [ 2.28192943e

In [17]:
best_structure_Au = np.array(pos_Au_final)[-1]
lowest_energy_Au = np.array(err_Au_final)[-1][-1]

In [22]:
print(best_structure_Au.shape)

(20, 3)


写入文件

In [25]:
filename = 'e:/Python_code/MathorCup/data/Au_20_best.xyz'
with open(filename, 'w') as f:
    f.write(str(best_structure_Au.shape[0]) + "\n")
    f.write(str(lowest_energy_Au) + "\n")
    for i in best_structure_Au:
        f.write("Au      " + str(i[0]) + "       " +  str(i[1]) + "       " +  str(i[2])  + "\n")
    
    # file_object.write("Add two words\n")
    f.close()

In [26]:
def normalization(filename):
    xyz = []
    with open(filename, 'r') as f:
        content = f.read()
        # print("1", content)
        contact = content.strip().split('\n')
        n = len(contact)
        # print("store", contact[:2])
        store = contact[:2]
        # print()
        # print(len(contact))
        # print("2", contact)

        for line in contact:
            if len(line.split("      ")) >= 4:
                # print("3", line.split("      "))
                line = line.split('      ')
                coord = [float(line[1]), float(line[2]), float(line[3])]
                # print(coord)
                xyz.append(coord)
        f.close()
    xyz = np.array(xyz)
    scale = np.max(np.abs(np.array(xyz)))
    xyz /= scale
    atom = []
    for i in xyz:
        new = "Au      " + str(i[0]) + "      " + str(i[1]) + "      " + str(i[2]) + "\n"
        atom.append(new)
    # print(atom[0])
    with open(filename, 'w') as f:
        f.write(store[0] + "\n")
        f.write(store[1] + "\n")
        for i in range(2,n):
            f.write(atom[i-2])
        f.close()

In [27]:
normalization("./data/Au_20_best.xyz")

## Generation of Au32 structure

In [40]:
num = 32*3
initial_Au32_struct = np.random.uniform(-10,10,num).reshape(32,3)
Au32_bounds = [(-10, 10), (-10, 10), (-10, 10)]

In [57]:
epochs = 10
pos_Au32_final = []
err_Au32_final = []
for epoch in range(epochs):
    pos_Au32_final_tmp = []
    err_Au32_final_tmp = []
    for i in range(initial_Au32_struct.shape[0]):
        if epoch == 0 and i == 0:
            new_Au_pos = initial_Au32_struct
        else:
            new_Au_pos = new_Au32_pos_
        pos_Au32_best_g_tmp, err_Au32_best_g_tmp = PSO(SC_energy_func, new_Au_pos[i], i, Au32_bounds, num_particles=20, maxiter=80).pso()
        new_Au32_pos_ = np.vstack((new_Au_pos[:i], np.vstack((pos_Au32_best_g_tmp, new_Au_pos[i+1:]))))
        pos_Au32_final_tmp.append(pos_Au32_best_g_tmp)
        err_Au32_final_tmp.append(err_Au32_best_g_tmp)
    pos_Au32_final.append(pos_Au32_final_tmp)
    err_Au32_final.append(err_Au32_final_tmp)

10, 20, 50  -6895.116880725851 \\
15, 15, 30  -6960.735413334635 \\
15, 20, 50  -6894.637502089301 \\
10, 20, 80  -7048.019096939294

In [69]:
# print("Best Au32 struction:\n", np.array(pos_Au32_final)[-1])
print("Lowest Au32 energy:\n", np.array(err_Au32_final)[-1][-1])

In [59]:
best_structure_Au32 = np.array(pos_Au32_final)[-1]
lowest_energy_Au32 = np.array(err_Au32_final)[-1][-1]
filename = 'e:/Python_code/MathorCup/data/Au32_best.xyz'
with open(filename, 'w') as f:
    f.write(str(best_structure_Au32.shape[0]) + "\n")
    f.write(str(lowest_energy_Au32) + "\n")
    for i in best_structure_Au32:
        f.write("Au      " + str(i[0]) + "       " +  str(i[1]) + "       " +  str(i[2])  + "\n")
    
    # file_object.write("Add two words\n")
    f.close()

In [60]:
normalization("./data/Au32_best.xyz")

## B45

In [4]:
def load_data_B45(file_dir):
    pos = []
    energy = []
    file_list = os.listdir(file_dir)
    for i1 in file_list:
        orig_os = os.getcwd()
        pos_tmp = []
        energy_tmp = []
        os.chdir(file_dir)    ## ./data/Au20_OPT_1000/
        with open(i1,'r') as df:
            dfread = df.read()
            match_tmp1 = re.findall(r"Energy:.*", dfread)
            # print(match_tmp1[0].split(":"))
            energy_tmp.append(float(match_tmp1[0].split(":")[1]))
            match_tmp2 = re.findall(r"\n.*B.*", dfread)
            # print(match_tmp2)
            for pop_i in range(len(match_tmp2)):
                xyz = match_tmp2[pop_i].split("       ")
                # print(xyz)
                x, y, z = float(xyz[1]), float(xyz[2]), float(xyz[3])
                pos_tmp.append([x, y, z])
        os.chdir(orig_os)
        pos = pos + pos_tmp
        energy = energy + energy_tmp
    pos = np.array(pos)
    energy = np.array(energy)
    return pos, energy

In [5]:
file_dir_B = 'e:/Python_code/MathorCup/data/B45-_OPT_3751'
pos_tmp_B, energy_B = load_data_B45(file_dir_B)
pos_B = pos_tmp_B.reshape(-1, 45, 3)

In [6]:
def SC_B_potential_energy_func(x, a):    ## 需要训练的势能函数
    n = 10
    m = 8
    c = 34.408
    # a = 1
    epsilon = 35

    dis_mat = np.array(dist_matric(x))
    tmp1 = (dis_mat / a + np.eye(dis_mat.shape[0])) ** (-n) - np.eye(dis_mat.shape[0])
    tmp2 = (dis_mat / a + np.eye(dis_mat.shape[0])) ** (-m) - np.eye(dis_mat.shape[0])
    vr = 1 / 2 * (np.sum(tmp1, axis=1))
    vd = -c * np.sqrt(np.sum(tmp2, axis=1))
    energy = epsilon * np.sum(vr + vd)
    return energy

In [7]:
def criterion(label,pred):
    return np.mean(np.abs(label - pred))

def parameter_selection(data, para_grid):
    error_total = []
    for para in para_grid:
        error_l = 0
        for i in range(data.shape[0]):
            ene = SC_B_potential_energy_func(data[i],para)
            err = criterion(energy_B[i],ene)
            error_l += err
        error_total.append(error_l)
    error_total = np.array(error_total)
    indx = np.argmin(error_total)
    return para_grid[indx], np.min(error_total/data.shape[0])

In [12]:
B_a = np.linspace(1, 3, 200)
best_B_a, error_B_train = parameter_selection(pos_B[:3000], B_a)
print("Best parameter for B45:", best_B_a)
print("Train error:", error_B_train)

Best parameter for B: 1.6231155778894473
Train error: 39813.97235646427


In [31]:
_, error_B_test = parameter_selection(pos_B[3000:], [best_B_a])
print("Test error:", error_B_test)

Test error: 34794.18483855465


In [34]:
def SC_B_energy_func(x, idx):
    n = 10
    m = 8
    c = 34.408
    a = best_B_a
    # epsilon = best_Au_epsilon
    epsilon = 35
    x = np.array(x)
    new_B_pos_ = np.vstack((new_B_pos[:idx], np.vstack((x, new_B_pos[idx+1:]))))
    new_dis = np.array(dist_matric(new_B_pos_))
    tmp1 = (new_dis / a + np.eye(new_dis.shape[0])) ** (-n) - np.eye(new_dis.shape[0])
    tmp2 = (new_dis / a + np.eye(new_dis.shape[0])) ** (-m) - np.eye(new_dis.shape[0])
    vr = 1 / 2 * (np.sum(tmp1, axis=1))
    vd = -c * np.sqrt(np.sum(tmp2, axis=1))
    energy = epsilon * np.sum(vr + vd)
    return energy

In [38]:
pos_B_init = pos_B[0]
new_B_pos = pos_B_init
energy_B_init = SC_energy_func(pos_B_init[0], 0)
print("orig pos:\n", pos_B_init)
print("orig energy:\n", energy_B_init)
print("real energy:\n", energy_B[0])

x_max_B = np.max(pos_B[:,:,0])
x_min_B = np.min(pos_B[:,:,0])
y_max_B = np.max(pos_B[:,:,1])
y_min_B = np.min(pos_B[:,:,1])
z_max_B = np.max(pos_B[:,:,2])
z_min_B = np.min(pos_B[:,:,2])
x_B_bounds = (x_min_B, x_max_B)
y_B_bounds = (y_min_B, y_max_B)
z_B_bounds = (z_min_B, z_max_B)
pos_B_final = []
err_B_final = []

B_bounds = [x_B_bounds, y_B_bounds, z_B_bounds]
epochs = 10
for epoch in range(epochs):
    pos_B_final_tmp = []
    err_B_final_tmp = []
    for i in range(pos_B_init.shape[0]):
        if epoch == 0 and i == 0:
            new_B_pos = pos_B_init
        else:
            new_B_pos = new_B_pos_
        pos_B_best_g_tmp, err_B_best_g_tmp = PSO(SC_B_energy_func, new_B_pos[i], i, B_bounds, num_particles=15, maxiter=30).pso()
        new_B_pos_ = np.vstack((new_B_pos[:i], np.vstack((pos_B_best_g_tmp, new_B_pos[i+1:]))))
        pos_B_final_tmp.append(pos_B_best_g_tmp)
        err_B_final_tmp.append(err_B_best_g_tmp)
    pos_B_final.append(pos_B_final_tmp)
    err_B_final.append(err_B_final_tmp)

orig pos:
 [[-2.50828e+00 -1.75993e+00 -4.87900e-01]
 [-2.42680e+00 -3.33574e+00  3.86300e-02]
 [-1.04626e+00 -4.15733e+00  1.22470e-01]
 [ 3.52190e-01 -3.38479e+00 -5.14560e-01]
 [-1.12871e+00 -8.48860e-01 -1.04054e+00]
 [ 1.78797e+00 -8.30370e-01 -9.13800e-01]
 [ 1.74938e+00 -4.14389e+00  2.46060e-01]
 [-3.95566e+00 -9.11200e-01 -2.15620e-01]
 [ 1.80277e+00 -2.53237e+00 -4.88970e-01]
 [-3.80894e+00 -2.53783e+00  2.92750e-01]
 [ 3.43160e-01 -4.74603e+00  3.51360e-01]
 [ 3.09279e+00 -3.27960e+00  2.66370e-01]
 [ 3.17357e+00 -1.67952e+00 -2.60190e-01]
 [ 4.53172e+00 -7.96520e-01  2.39490e-01]
 [ 3.12080e-01  1.83058e+00 -1.08665e+00]
 [ 1.72326e+00  2.55165e+00 -4.29270e-01]
 [ 3.12807e+00  1.73388e+00 -2.19660e-01]
 [ 4.54706e+00  8.95460e-01  1.43370e-01]
 [ 4.32534e+00  2.50059e+00  7.70190e-01]
 [ 3.00756e+00  3.34177e+00  2.78400e-01]
 [ 1.64059e+00  4.17036e+00  2.58000e-01]
 [ 2.67960e-01  3.40327e+00 -5.39570e-01]
 [ 2.18090e-01  4.74820e+00  3.52430e-01]
 [-2.51497e+00  3.30378

In [40]:
print("Best struction for B45:\n", np.array(pos_B_final)[-1])
print("Lowest energy for B45:\n", np.array(err_B_final)[-1][-1])

Best struction for B45:
 [[-1.20498344e+00 -2.68771575e+00 -9.98928375e-02]
 [-3.37009502e+00 -1.55320392e+00  1.89698600e-01]
 [-3.40799688e-01 -2.56064652e+00 -5.79520280e-01]
 [ 1.12641605e+00 -2.57262852e+00 -1.21828125e+00]
 [-1.20059918e-01  1.03075513e+00 -1.05323887e+00]
 [ 2.66389264e+00 -2.50528113e+00 -1.80492262e-01]
 [ 2.17463892e+00 -3.39351122e+00 -1.02445281e-01]
 [-4.27718084e+00 -1.56200776e+00  6.09714834e-01]
 [ 1.61947838e+00 -2.52543527e+00 -2.97526899e-01]
 [-3.70988213e+00 -2.37895150e+00  6.09031865e-01]
 [ 6.02913694e-01 -2.87021467e+00 -3.79574755e-01]
 [ 3.16365287e+00 -1.66116838e+00  1.10686667e-01]
 [ 4.19194620e+00 -1.59753660e+00  1.35146660e-01]
 [ 3.60617009e+00 -7.13216462e-01  2.95038539e-02]
 [-1.43614511e+00  3.04576073e+00 -4.12186294e-01]
 [ 2.67820782e+00  2.55129405e+00  3.37958122e-01]
 [ 4.02631585e+00  1.64929617e+00  2.51207874e-01]
 [ 4.19369349e+00  1.74348982e+00  1.29419412e+00]
 [ 3.57260799e+00  2.39801773e+00  8.06442111e-01]
 [ 2.4

In [41]:
best_structure_B45 = np.array(pos_B_final)[-1]
lowest_energy_B45 = np.array(err_B_final)[-1][-1]

In [48]:
filename = 'e:/Python_code/MathorCup/data/B45_best.xyz'
with open(filename, 'w') as f:
    f.write(str(best_structure_B45.shape[0]) + "\n")
    f.write(str(lowest_energy_B45) + "\n")
    for i in best_structure_B45:
        f.write("B       " + str(i[0]) + "       " +  str(i[1]) + "       " +  str(i[2])  + "\n")
    
    # file_object.write("Add two words\n")
    f.close()

In [49]:
def normalization_B(filename):
    xyz = []
    with open(filename, 'r') as f:
        content = f.read()
        # print("1", content)
        contact = content.strip().split('\n')
        n = len(contact)
        # print("store", contact[:2])
        store = contact[:2]
        # print()
        # print(len(contact))
        # print("2", contact)

        for line in contact:
            if len(line.split("       ")) >= 4:
                # print("3", line.split("      "))
                line = line.split('       ')
                coord = [float(line[1]), float(line[2]), float(line[3])]
                # print(coord)
                xyz.append(coord)
        f.close()
    xyz = np.array(xyz)
    scale = np.max(np.abs(np.array(xyz)))
    xyz /= scale
    atom = []
    for i in xyz:
        new = "B       " + str(i[0]) + "       " + str(i[1]) + "       " + str(i[2]) + "\n"
        atom.append(new)
    # print(atom[0])
    with open(filename, 'w') as f:
        f.write(store[0] + "\n")
        f.write(store[1] + "\n")
        for i in range(2,n):
            f.write(atom[i-2])
        f.close()

In [50]:
normalization_B("e:/Python_code/MathorCup/data/B45_best.xyz")

## Generation of B40 structure

In [53]:
B_bounds

[(-6.90263, 7.07114), (-5.11455, 5.48891), (-2.79003, 3.75261)]

In [56]:
num_B40 = 40*3
initial_B40_struct = np.random.uniform(-7,7,num_B40).reshape(40,3)
B40_bounds = [(-7, 7), (-7, 7), (-7, 7)]

In [57]:
epochs = 10
pos_B40_final = []
err_B40_final = []
for epoch in range(epochs):
    pos_B40_final_tmp = []
    err_B40_final_tmp = []
    for i in range(initial_B40_struct.shape[0]):
        if epoch == 0 and i == 0:
            new_B_pos = initial_B40_struct
        else:
            new_B_pos = new_B40_pos_
        pos_B40_best_g_tmp, err_B40_best_g_tmp = PSO(SC_B_energy_func, new_B_pos[i], i, B40_bounds, num_particles=20, maxiter=80).pso()
        new_B40_pos_ = np.vstack((new_B_pos[:i], np.vstack((pos_B40_best_g_tmp, new_B_pos[i+1:]))))
        pos_B40_final_tmp.append(pos_B40_best_g_tmp)
        err_B40_final_tmp.append(err_B40_best_g_tmp)
    pos_B40_final.append(pos_B40_final_tmp)
    err_B40_final.append(err_B40_final_tmp)

In [ ]:
print("Best B40 struction:\n", np.array(pos_B40_final)[-1])
print("Lowest B40 energy:\n", np.array(err_B40_final)[-1][-1])

In [ ]:
best_structure_B40 = np.array(pos_B40_final)[-1]
lowest_energy_B40 = np.array(err_B40_final)[-1][-1]
filename = 'e:/Python_code/MathorCup/data/B40_best.xyz'
with open(filename, 'w') as f:
    f.write(str(best_structure_B40.shape[0]) + "\n")
    f.write(str(lowest_energy_B40) + "\n")
    for i in best_structure_B40:
        f.write("B       " + str(i[0]) + "       " +  str(i[1]) + "       " +  str(i[2])  + "\n")
    
    # file_object.write("Add two words\n")
    f.close()

In [ ]:
normalization("./data/Au32_best.xyz")